In [2]:
!pwd

/Users/hugocoussens/git/betfair_profitbox/research


In [30]:
from pathlib import Path
import json, csv
from pathlib import Path


paths = [str(p) for p in Path("./hist_data/ADVANCED/").rglob("*") if p.is_file()]
paths = [p for p in paths if not p.lower().endswith(".ds_store")]
matches = []

for path in paths:
    with open(path, "r") as f:
        try:
            for line in f:
                if '"marketType":"MATCH_ODDS"' in line and '"status":"ACTIVE"' and '"name":"Match Odds"' in line:
                    matches.append(line.strip())
        except: print(path)


lines = matches  # your list of JSON strings

def safe_load_json(s):
    try: return json.loads(s)
    except: return json.loads(s[s.index("{"):])

unique = {}
for ln in lines:
    o = safe_load_json(ln)
    for mc in o.get("mc", []):
        md = mc.get("marketDefinition", {})
        k = (md.get("eventId"), mc.get("id"))
        if all(k):
            if md["runners"][0]["status"] == 'ACTIVE' and len(md["runners"]) ==2 and md["runners"][0]["name"] not in ("No", "Yes"):
                runners = (md["runners"])
                unique[k] = {"eventName": md.get("eventName"), "id": mc.get("id"), "eventId": md.get("eventId"), "runners": runners}

with open("input.csv","w",newline="",encoding="utf-8") as f:
    csv.DictWriter(f,fieldnames=["eventName","id","eventId","runners"]).writeheader(); f.writelines(
        ",".join(map(str,row.values()))+"\n" for row in unique.values()
    )

 # Gets all MATCH_ODDS event and market ids into csv brute force  ^^^


files = {Path(p).name: p for p in paths}
nlines = lambda p: sum(1 for _ in open(p, "rb"))

with open("input.csv", encoding="utf-8") as f:
    header = f.readline().rstrip("\n")
    h1,h2,h3,_ = header.split(",", 3)  # eventName,id,eventId,runners
    lines = [ln.rstrip("\n") for ln in f]

bad = set()
for ln in lines:
    try:
        _, idv, _, _ = ln.split(",", 3)
        fp = files.get(idv)
        if fp and nlines(fp) < 200:
            bad.add(idv)
    except ValueError:
        continue  # skip malformed rows

with open("bought_data_catalogue.csv", "w", encoding="utf-8") as f:
    f.write(header + "\n")
    for ln in lines:
        try:
            _, idv, _, _ = ln.split(",", 3)
            if idv not in bad:
                f.write(ln + "\n")
        except ValueError:
            pass

 # Cleans up to only have market ids with >200 lines (aka more likely the actual match odds) - wouldnt work on more granual tick data files)

with open("bought_data_catalogue.csv", newline="", encoding="utf-8") as f:
    reader = csv.reader(f)
    rows = list(reader)
with open("bought_data_catalogue.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    for count, row in enumerate(rows):
        if count >0:
            row.append([x for x in files.values() if row[1] in x][0])
            writer.writerow(row)
        else:
            row.append("path")
            writer.writerow(row)

!rm input.csv